### instalando e importando o bark

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install git+https://github.com/suno-ai/bark.git -q
!pip install scipy -q

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 598.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 118.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 122.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
import scipy
import os

# Este import agora vai funcionar sem o OSError


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

# Parte 1 - Extração da Letra da música e o instrumental

## Copie o caminho do arquivo

In [ ]:
!pip install numpy==1.26.4

In [ ]:
!pip install demucs
!python -m demucs.separate "/content/Beat_it.wav"

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track /content/Beat_it.wav
100%|██████████████████████████████████████████████████████████████████████| 263.25/263.25 [00:12<00:00, 21.34seconds/s]


# Parte 2 - Tradução da letra da música

In [ ]:
!pip install openai pyphen
!pip install -U openai-whisper -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import openai
import pyphen
import whisper

client = openai.OpenAI(api_key=                                                                                                                                                  "")
dic = pyphen.Pyphen(lang='pt_BR')
dic_pt = pyphen.Pyphen(lang='pt_BR')
dic_en = pyphen.Pyphen(lang='en_US')


### Parte 2.1: Extraindo a letra da música

In [ ]:
import os
import whisper

base_dir = "/content/separated/htdemucs"

# Lista as subpastas dentro do diretório base
subdirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

if len(subdirs) == 0:
    raise FileNotFoundError(f"Nenhuma subpasta encontrada em {base_dir}")

# Pega a primeira subpasta encontrada (ex: 'test' ou 'beat_it')
subfolder = subdirs[0]

# Monta o caminho completo para o arquivo vocals.wav
vocals_path = os.path.join(base_dir, subfolder, "vocals.wav")

print(f"Carregando áudio de: {vocals_path}")

Carregando áudio de: /content/separated/htdemucs/Beat_it/vocals.wav


In [ ]:
# Carrega o áudio usando whisper
voz = whisper.load_audio(vocals_path)
model = whisper.load_model("medium")
result = model.transcribe(voz, verbose=True)

100%|█████████████████████████████████████| 1.42G/1.42G [00:25<00:00, 61.1MiB/s]
/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:30.000 --> 00:43.880]  They told them don't you ever come around here Don't wanna see your face, you better
[00:43.880 --> 00:47.320]  disappear The fire's in their eyes and their words are
[00:47.320 --> 00:52.480]  really clear So beat it, just beat it
[00:52.480 --> 00:57.640]  You better run, you better do what you can Don't wanna see no blood, don't be a macho
[00:57.640 --> 00:58.640]  man
[00:58.800 --> 01:06.200]  You wanna be tough, better do what you can So beat it, but you wanna be fair, just beat
[01:06.200 --> 01:12.560]  it, beat it, beat it No one wants to beat it, beat it
[01:12.560 --> 01:19.280]  Showin' hot funky, strong, we shall fight it It doesn't matter who's woke up right,
[01:19.280 --> 01:24.080]  just beat it Just beat it, just beat it
[01:24.080 --> 01:29.840]  There are things that you better leave while you can
[01:29.840 --> 01:36.5

In [ ]:
letra_extraida = result["text"]
print("\n✅ Letra extraída pelo Whisper:")
print(letra_extraida)


✅ Letra extraída pelo Whisper:
 They told them don't you ever come around here Don't wanna see your face, you better disappear The fire's in their eyes and their words are really clear So beat it, just beat it You better run, you better do what you can Don't wanna see no blood, don't be a macho man You wanna be tough, better do what you can So beat it, but you wanna be fair, just beat it, beat it, beat it No one wants to beat it, beat it Showin' hot funky, strong, we shall fight it It doesn't matter who's woke up right, just beat it Just beat it, just beat it There are things that you better leave while you can Don't wanna be a boy, you wanna be a man You wanna stay alive, better do what you can So beat it, just beat it You have to show them that you're really not scared You're playin' with your life, this ain't no truth or dare They'll kick you, they'll beat you, they'll tell you it's fair So beat it, but you wanna be bad Just beat it, beat it No one wants to beat it, beat it Showin'

### Parte 2.2: Traduzindo a letra

In [ ]:
def contar_silabas_verso(verso: str) -> int:
    palavras = verso.split()
    silabas = sum(len(dic.inserted(palavra).split('-')) for palavra in palavras)
    return silabas

In [ ]:
def processar_letra(letra_original: str):
    print("🎵 Traduzindo e adaptando...\n")
    letra_traduzida = traduzir_com_adaptacao(letra_original)

    print("✅ Letra traduzida e adaptada:")
    print("-" * 50)
    print(letra_traduzida)
    print("-" * 50)

    print("\n📏 Contando sílabas poéticas por verso:")
    for linha in letra_traduzida.splitlines():
        if linha.strip():
            silabas = contar_silabas_verso(linha)
            print(f"{linha}  →  {silabas} sílabas")

In [ ]:
def traduzir_com_adaptacao(letra_original: str, destino="português do Brasil") -> str:
    prompt = f"""
    Traduza e adapte a seguinte letra de música para o {destino}.

    Requisitos:
    - Preserve a emoção original.
    - Mantenha a métrica (número de sílabas poéticas por verso) o mais próximo possível.
    - Preserve rimas sempre que possível.
    - Formate a letra com uma quebra de linha por verso (cada verso em uma linha).
    - Agrupe versos em estrofes separadas por uma linha em branco.

    Letra original:
    {letra_original}
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9
    )
    return response.choices[0].message.content

In [ ]:
letra_traduzida_portugues = traduzir_com_adaptacao(letra_extraida)

print("✅ Letra traduzida e adaptada:\n")

print(letra_traduzida_portugues)


✅ Letra traduzida e adaptada:

Eles disseram, não apareça por aqui  
Não quero ver seu rosto, é melhor sumir  
Há fogo em seus olhos, palavras claras a gritar  
Então caia fora, é melhor nem tentar  

É melhor correr, fazer o que puder  
Não quero ver sangue, não seja durão  
Quer ser forte, melhor fazer o que puder  
Então caia fora, mas queira ser justo, só vá, vá  

Ninguém quer lutar, lutar  
Mostre sua força, vamos enfrentar  
Não importa quem está certo ou errado, só vá  
Só vá, só vá  

Há coisas que você deve deixar pra trás  
Não queira ser menino, queira ser capaz  
Quer sobreviver, faça o que puder  
Então caia fora, só vá embora  

Mostre que não está com medo de verdade  
Está brincando com a vida, não é só vaidade  
Eles vão chutar, vão bater, vão dizer que é justo  
Então caia fora, mas quer ser durão  

Só vá, vá  
Ninguém quer lutar, lutar  
Mostre sua força, vamos enfrentar  
Não importa quem está certo ou errado  

Só vá  
Só vá  
Ninguém quer lutar, lutar  
Mostre s

In [ ]:
# Salva a letra traduzida limpa
letra = letra_traduzida_portugues

with open("letra_para_diffsinger.txt", "w", encoding="utf-8") as f:
    for linha in letra.splitlines():
        if linha.strip():
            f.write(linha.strip() + "\n")

# 1. Instalação de dependências
!git clone https://github.com/openvpi/DiffSinger
%cd DiffSinger
!pip install -r requirements.txt
!pip install praat-parselmouth pyworld phonemizer

# 2. Preparação de arquivos
from pathlib import Path
import shutil

# Caminhos dos seus arquivos
midi_path = "/content/BeatIt.mid"
voz_ref_path = "/content/separated/htdemucs/beat_it/vocals.wav"

# Criação de diretório para os dados
Path("raw/dataset/test_song").mkdir(parents=True, exist_ok=True)

# Copia os arquivos
shutil.copy("letra_para_diffsinger.txt", "raw/dataset/test_song/lyrics.txt")
shutil.copy(midi_path, "raw/dataset/test_song/notes.mid")
shutil.copy(voz_ref_path, "raw/dataset/test_song/gt.wav")


Cloning into 'DiffSinger'...
remote: Enumerating objects: 9009, done.
remote: Counting objects: 100% (431/431), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 9009 (delta 318), reused 247 (delta 228), pack-reused 8578 (from 2)
Receiving objects: 100% (9009/9009), 67.79 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (5570/5570), done.
/content/DiffSinger
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 24.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 106.5 MB/s eta 0:00:00
   ━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 45.7 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: 'letra_para_diffsinger.txt'

# Parte 3 - Ajuste da métrica e da melodia

# Parte 4 - Geração da voz cantada na nova lingua

##### movi os imports do bark para o começo

In [ ]:
preload_models()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 109MB/s]


### usando a letra gerada

In [ ]:
letra = """
Você partiu meu coração
Mas eu segui em frente, então
Agora eu canto essa canção
Com alma, voz e emoção
"""

In [ ]:
texto_para_bark = "♪ " + letra_traduzida_portugues.replace("\n", " ... ") + " ♪"
audio_array = generate_audio(texto_para_bark, history_prompt="v2/pt_speaker_0") # "pt" para voz em português

100%|██████████| 37/37 [00:34<00:00,  1.06it/s]


In [ ]:
caminho_novo_vocal = "novo_vocal_gerado.wav"
scipy.io.wavfile.write(caminho_novo_vocal, rate=SAMPLE_RATE, data=audio_array)

print(f"✅ Novo vocal gerado e salvo em: {caminho_novo_vocal}")

✅ Novo vocal gerado e salvo em: novo_vocal_gerado.wav
